In [2]:
from matplotlib import markers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from glob import glob
from seaborn import palettes
from seaborn.categorical import swarmplot
from tqdm import tqdm
import json
from scipy.signal import detrend
from scipy.ndimage import gaussian_filter1d
import os
import re
import math
import matplotlib.ticker as tck
import glob
import argparse
import statsmodels.api as sm


Loading json files wtih cell info

In [3]:
paths = glob.glob(os.path.join("E:/Turku_collab/lunagel_50s/remodelling", "**", "track.json"), recursive=True)
if not paths:
    print("No files found")
else:
    for path in paths:
        print(path)

E:/Turku_collab/lunagel_50s/remodelling\24_120\120\0_241111\241111_10um_3layer_50s_MDAMB231\24111101010201_UV\track.json
E:/Turku_collab/lunagel_50s/remodelling\24_120\120\0_241111\241111_10um_3layer_50s_MDAMB231\24111101010101_UV\track.json
E:/Turku_collab/lunagel_50s/remodelling\24_120\120\0_241111\241111_10um_3layer_50s_MDAMB231\24111101010301_UV\track.json
E:/Turku_collab/lunagel_50s/remodelling\24_120\120\0_241111\241111_10um_3layer_50s_MDAMB231\24111101010401_UV\track.json
E:/Turku_collab/lunagel_50s/remodelling\24_120\120\0_241111\241111_10um_3layer_50s_MDAMB231\24111101010501_UV\track.json
E:/Turku_collab/lunagel_50s/remodelling\24_120\120\0_241111\241111_10um_3layer_50s_MDAMB231\24111101010601_UV\track.json
E:/Turku_collab/lunagel_50s/remodelling\24_120\120\0_241111\241111_10um_3layer_50s_control\24111101010401_UV\track.json
E:/Turku_collab/lunagel_50s/remodelling\24_120\120\0_241111\241111_10um_3layer_50s_control\24111101010101_UV\track.json
E:/Turku_collab/lunagel_50s/remode

Extracting cell info file with sample info & cell location and size

In [4]:
# cells move 0.6µm/min --> in 6 hours = 216µm
# hamamatsu_zoom pixel size = 0.325µm
# 664,6 radius

df_list = []
for i in paths:
    data = pd.read_json(i)
    
    keys_list = [key for key in data.keys() if 'cell' in key]
    idents = os.path.normpath(i).split(os.sep)
    
    for key in keys_list:
        id = key.split('_')[1]
        new = {
        'incubation_set': idents[4],
        "incubation": idents[5],
        "experiment": idents[6].split('_')[0],       
        "day": idents[-2][0:6],
        "sample": idents[-3].split('_')[-1],
        "holder": idents[-2][9:10],
        "location": idents[-2][11:12],
        "sample_type": idents[-2][15:17],
        "cell_id": id,
        "x":data[key]['x'][1],
        "y":data[key]['y'][1],
        "size":data[key]['radius'][5],
        }
        
        df_list.append(new)
        
final = pd.DataFrame(df_list)
print(final)

final.to_csv(os.path.join("E:/Turku_collab/lunagel_50s/remodelling", "cell_info.csv"))


    incubation_set incubation experiment     day    sample holder location  \
0           24_120        120          0  241111  MDAMB231      1        2   
1           24_120        120          0  241111  MDAMB231      1        2   
2           24_120        120          0  241111  MDAMB231      1        2   
3           24_120        120          0  241111  MDAMB231      1        2   
4           24_120        120          0  241111  MDAMB231      1        2   
..             ...        ...        ...     ...       ...    ...      ...   
930         48_144        144          1  241126  MDAMB231      1        6   
931         48_144        144          1  241126  MDAMB231      1        6   
932         48_144        144          1  241126  MDAMB231      1        6   
933         48_144        144          1  241126  MDAMB231      1        6   
934         48_144        144          1  241126  MDAMB231      1        6   

    sample_type cell_id       x       y       size  
0         